Importaciones:

In [8]:
import cv2  
import math 
import pytesseract
import re

from ultralytics import YOLO

Verificar si la ruta de Tesseract está en el PATH

In [9]:
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
detector = r'\d+.[A-Z]+'

Activación de la cámara y detección de matrículas:

# Modelo Yolo

In [ ]:
# Carga del modelo
model = YOLO('yolov8n.pt')

# Captura desde lawebcam
vid = cv2.VideoCapture(0)

while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)

        # Para cada detección
        for r in results:
            boxes = r.boxes
            for box in boxes:
                if int(box.cls[0]) == 2:
                    # Contenedor
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                    # Dibuja el contenedor del coche
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)

                    # Dibuja el contenedor de la mitad inferior del coche
                    cv2.rectangle(img, (x1, y1+(y2-y1)//2), (x2, y2), (255, 0, 0), 3)
                   
                    # Extraemos el fragmento de la matrícula
                    slice = img[y1+(y2-y1)//2:y2,x1:x2]

                    #cv2.imshow('slice', slice)

                    # Lo convertimos a RGB
                    img_rgb = cv2.cvtColor(slice, cv2.COLOR_BGR2RGB)

                    # Detectamos el texto del fragmento
                    text = pytesseract.image_to_string(img_rgb, config='--psm 6')
                    matricula_limpia = str(re.findall(detector, text))
                    cv2.putText(img, matricula_limpia , [x1, y1+(y2-y1)//2], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

In [11]:
# Tesseract
import cv2
import pytesseract

# Previamente debes descargar los ejecutables
# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Lenguajes disponibles
print(pytesseract.get_languages(config=''))

#Cargo imagen y ocnvierto a RGB
img = cv2.imread('c.jpeg') 
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Aplica reconocedor a imagen cargada
text = pytesseract.image_to_string(img_rgb,config='--psm 6')
matricula_limpia = re.findall(detector, text)
print("Texto detctado:", text)
print("Texto limpio:", matricula_limpia)



['eng', 'osd']
Texto detctado: 7 am. v se

‘a a
BS BO411 KMT R=!
Scere ee TT mi

Texto limpio: ['411 KMT']


# Modelo entrenado


In [ ]:
# Carga del modelo
model = YOLO('./best.pt')

# Captura desde lawebcam
vid = cv2.VideoCapture(0)

while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Perform inference on an image
        results = model(img, stream=True)

        # Para cada detección
        for r in results:
            boxes = r.boxes
            for box in boxes:
                
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                # Dibuja el contenedor de la matricula
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                   
                # Extraemos el fragmento de la matrícula
                slice = img[y1:y2,x1:x2]

                #cv2.imshow('slice', slice)

                # Lo convertimos a RGB
                img_rgb = cv2.cvtColor(slice, cv2.COLOR_BGR2RGB)

                # Detectamos el texto del fragmento
                text = pytesseract.image_to_string(img_rgb, config='--psm 6')
                matricula_limpia= str(re.findall(detector, text))
                cv2.putText(img, matricula_limpia , [x1, y1-20], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
        # Muestra fotograma
        cv2.imshow('Vid', img)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()